## 금주 목표
### 이번 시즌 팀 순위 시각화

In [202]:
import natsort as natsort
import pandas as pd
import os
import natsort
from IPython.core.display_functions import display

In [40]:
for month in range(4, 10):
    url = "http://www.statiz.co.kr/schedule.php?opt={}&sy=2022".format(month)
    print(url)
df = pd.read_html(url)
schedule = df[1]

http://www.statiz.co.kr/schedule.php?opt=4&sy=2022
http://www.statiz.co.kr/schedule.php?opt=5&sy=2022
http://www.statiz.co.kr/schedule.php?opt=6&sy=2022
http://www.statiz.co.kr/schedule.php?opt=7&sy=2022
http://www.statiz.co.kr/schedule.php?opt=8&sy=2022
http://www.statiz.co.kr/schedule.php?opt=9&sy=2022


In [98]:
month = 4
url = "https://sports.news.naver.com/kbaseball/schedule/index?date=20220927&month={}&year=2022&teamCode=".format(month)
df = pd.read_html(url)

In [119]:
for i in range(1, len(df)-1): #제일 위에 column 목차가 df단위로 떨어져서 삭제하는걸로 함, 그리고 마지막 df는 <맨위로>키 정보가 있는 테이블이라 범위에 넣지 않음.
    if df[i][2][0] == "프로야구 경기가 없습니다.":
        continue
    remain= df[i].loc[:, [0, 1, 2, 4, 5]]
    remain.rename(columns={0:"일자", 1:"시각", 2:"경기결과", 4:"중계사", 5:"구장"} , inplace=True)
    text = remain["일자"][0]
    remain.to_csv("./date/"+text, header=True, encoding='UTF-8')

In [221]:
for month in range(4, 10):
    url = "https://sports.news.naver.com/kbaseball/schedule/index?&month={}&year=2022&teamCode=".format(month)
    df = pd.read_html(url)
    for i in range(1, len(df)-1): #제일 위에 column 목차가 df단위로 떨어져서 삭제하는걸로 함, 그리고 마지막 df는 <맨위로>키 정보가 있는 테이블이라 범위에 넣지 않음.
        if df[i][2][0] == "프로야구 경기가 없습니다.":
            continue
        remain= df[i].loc[:, [0, 1, 2, 4, 5]]
        remain.rename(columns={0:"일자", 1:"시각", 2:"경기결과", 4:"중계사", 5:"구장"} , inplace=True)
        text = remain["일자"][0].split()[0]
        remain.to_csv("./date/"+text+'.csv', header=True, encoding='UTF-8', index=False)
    print(month, "완료")

4 완료
5 완료
6 완료
7 완료
8 완료
9 완료


## DataFrame 에서 얻어낸 것
1. 네이버는 https 프로토콜을 사용하므로 ssl때문에 certification 관련 추가 작업이 필요했음
1-1. 따라서 terminal에 `pip install --upgrade certifi`를 추가적으로 작성함
1-2. 그리고 Application > python 3.10 > install certifi 진행
2. 네이버의 경우, 일정별로 깔끔하게 떨어져서 나옴
3. 다만, 데이터프레임의 개수가 많이 나타남

In [222]:
start_data = {"LG":[0,0,0,0], "KIA":[0,0,0,0], "NC":[0,0,0,0], "SSG":[0,0,0,0], "한화":[0,0,0,0], "두산":[0,0,0,0],"키움":[0,0,0,0],"삼성":[0,0,0,0],"KT":[0,0,0,0], "롯데":[0,0,0,0]}
KBO_team = pd.DataFrame(start_data, index= ["경기수", "승리", "무승부","패배"])
KBO_team = KBO_team.T
KBO_team

,경기수,승리,무승부,패배
LG,0,0,0,0
KIA,0,0,0,0
NC,0,0,0,0
SSG,0,0,0,0
한화,0,0,0,0
두산,0,0,0,0
키움,0,0,0,0
삼성,0,0,0,0
KT,0,0,0,0
롯데,0,0,0,0


In [223]:
order_list = os.listdir("./date")
sorted_list = natsort.natsorted(order_list)

In [224]:
for file in sorted_list:
    data = pd.read_csv("./date/" + file)
    for i in data.index:
        line = data.loc[i, :]

        if len(line[2].split(":")) == 1:
            continue

        away, home = line[2].split(":")
        home = home.split()
        away = away.split()
        h_score, h_team = home
        a_team, a_score = away

        if h_team == "나눔" or a_team == "나눔":
            continue #올스타전 제외

        #리팩토링은 항상 모두 완료하고나서 할 것
        if h_score > a_score: #홈팀 승리
            KBO_team.loc[h_team, ["승리"]] +=1
            KBO_team.loc[a_team, ["패배"]] +=1
        elif h_score < a_score:
            KBO_team.loc[a_team, ["승리"]] +=1
            KBO_team.loc[h_team, ["패배"]] +=1
        else:
            KBO_team.loc[h_team, ["무승부"]] +=1
            KBO_team.loc[a_team, ["무승부"]] +=1

In [225]:
KBO_team["경기수"] = KBO_team["승리"] + KBO_team["패배"] + KBO_team["무승부"]
KBO_team.sort_values(by="승리", inplace=True, ascending=False)
KBO_team

# 경기 수는 맞는데, 승리가 이상하게 집계됨
# 원인은 아직 모르겠음

,경기수,승리,무승부,패배
SSG,137,85,4,48
키움,139,77,2,60
LG,132,69,2,61
KT,135,69,2,64
롯데,138,66,4,68
삼성,135,65,2,68
NC,133,64,3,66
KIA,136,62,1,73
두산,133,55,2,76
한화,136,53,2,81


In [232]:
ori_data  = {"LG":[132,81,2,49], "KIA":[136,65,1,70], "NC":[133,60,3,70], "SSG":[137,86,4,47], "한화":[136,44,2,90], "두산":[133,56,2,75],"키움":[139,78,2,59],"삼성":[135,60,2,73],"KT":[135,74,2,59], "롯데":[138,61,4,73]}

ori_team = pd.DataFrame(ori_data, index= ["경기수", "승리", "무승부","패배"])
ori_team = ori_team.T
ori_team.sort_values(by="승리", inplace=True, ascending=False)
ori_team

,경기수,승리,무승부,패배
SSG,137,86,4,47
LG,132,81,2,49
키움,139,78,2,59
KT,135,74,2,59
KIA,136,65,1,70
롯데,138,61,4,73
NC,133,60,3,70
삼성,135,60,2,73
두산,133,56,2,75
한화,136,44,2,90


In [233]:
KBO_team - ori_team
# 원본 데이터는 문제가 없음.
# 직접 수기로 세아린 결과, 정확히 똑같은 승리를 세아림
# 크롤링 한 것도 확인해봤는데, 문제 없었음.
# 아마 승, 패 넣는 과정에서 오류 생긴 것으로 보임

,경기수,승리,무승부,패배
KIA,0,-3,0,3
KT,0,-5,0,5
LG,0,-12,0,12
NC,0,4,0,-4
SSG,0,-1,0,1
두산,0,-1,0,1
롯데,0,5,0,-5
삼성,0,5,0,-5
키움,0,-1,0,1
한화,0,9,0,-9
